# Trabajo final del Diplomado en AI & DL Universidad Nacional de Colombia
## Clasificación de cambios de coberturas vegetales en imágenes de satélite Sentinel-2 en los humedales de Bogotá
### Fecha: Mayo de 2021
### Autores: Samuel Mesa, Wilgen Correa

### Resumen

Llevar a cabo la clasificación de coberturas vegetales a partir de imágenes de satélite SENTINEL-2 con aprendizaje profundo para la identificación de cambios de coberturas vegetales en los humedales de Bogotá.

In [1]:
import ee
ee.Initialize()

In [2]:
import geemap
from geetools import algorithms, ui, cloud_mask, tools
from pysatelital.utilidad import *
import folium

Definimos lass extensión geográfica de la zona de estudio

In [3]:
region = ee.Geometry.Rectangle(-74.149852,4.578917,-74.136691,4.567453)
tunjo = ee.FeatureCollection('users/femesagi/humedales/tunjo')
tunjo_buffer = region.buffer(500)

Búsqueda de las imágenes en una colección de imágenes de GEE con los parámetros de filtrado  de fecha y porcentaje de nubosidad. 

In [4]:
s2collection = ee.ImageCollection("COPERNICUS/S2_SR")\
    .filterBounds(tunjo_buffer)\
    .filterDate("2017-03-28", "2021-05-30")\
    .filterMetadata('CLOUD_COVERAGE_ASSESSMENT', 'less_than', 60)\
    .map(computeClouds)\
    .map(cloudsAtRegion(tunjo_buffer))

Listar las imágenes de la colección de imágenes

In [5]:
noclouds_s2collection = s2collection.filterMetadata('CLOUDS_AT_REGION', 'equals', 0)
#noclouds_s2collection = s2cloudcollection.filterMetadata('CLOUDS_AT_AOI', 'less_than', 10)
imagecol_masked = noclouds_s2collection.map(cloud_mask.sentinel2())
print(imagecol_masked.size().getInfo())
[img['id'] for img in  imagecol_masked.toList(imagecol_masked.size()).getInfo()]

47


['COPERNICUS/S2_SR/20181215T152631_20181215T153104_T18NWL',
 'COPERNICUS/S2_SR/20181220T152639_20181220T152634_T18NWL',
 'COPERNICUS/S2_SR/20181225T152631_20181225T153106_T18NWL',
 'COPERNICUS/S2_SR/20181230T152639_20181230T152636_T18NWL',
 'COPERNICUS/S2_SR/20190104T152631_20190104T153031_T18NWL',
 'COPERNICUS/S2_SR/20190124T152641_20190124T153110_T18NWL',
 'COPERNICUS/S2_SR/20190414T152641_20190414T152811_T18NWL',
 'COPERNICUS/S2_SR/20190807T152649_20190807T152646_T18NWL',
 'COPERNICUS/S2_SR/20190812T152641_20190812T152642_T18NWL',
 'COPERNICUS/S2_SR/20190911T152641_20190911T152637_T18NWL',
 'COPERNICUS/S2_SR/20191205T152639_20191205T153107_T18NWL',
 'COPERNICUS/S2_SR/20191210T152641_20191210T153058_T18NWL',
 'COPERNICUS/S2_SR/20191220T152631_20191220T153056_T18NWL',
 'COPERNICUS/S2_SR/20200104T152639_20200104T153054_T18NWL',
 'COPERNICUS/S2_SR/20200109T152631_20200109T152633_T18NWL',
 'COPERNICUS/S2_SR/20200114T152639_20200114T152633_T18NWL',
 'COPERNICUS/S2_SR/20200129T152631_20200

Seleccionar la última de las imágenes de GEE

In [8]:
listOfImages = noclouds_s2collection.toList(noclouds_s2collection.size())
ultima_imagen = listOfImages.size().getInfo() - 1
test_image = ee.Image(listOfImages.get(ultima_imagen))
print(test_image.id().getInfo())

20210319T152639_20210319T152638_T18NWL


In [9]:
mi_mapa = folium.Map(
    location=[4.575, -74.145],
    zoom_start=16,
    tiles='Stamen Terrain'
)

#mi_mapa.add_ee_layer(test_image, cloudVisParam, 'Whole Scene Cloudiness')
#mi_mapa.add_ee_layer(masked, imageVisParam, 'masked')
mi_mapa.add_ee_layer(test_image.clip(region), imageVisParam, 'Recorte de la zona de estudio')
#mi_mapa.add_ee_layer(region, {'color': 'red', 'strokeWidth': 15}, "Tunjo")
#mi_mapa.add_ee_layer(test_image, imageVisParam , 'Sential NDVI')
display(mi_mapa)